In [1]:
from keras.models import Model
from keras.layers import Input,Conv2D, Conv3D, MaxPooling3D,UpSampling2D, UpSampling3D,MaxPooling2D, concatenate, Conv3DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from keras.layers import Activation, MaxPool2D, Concatenate,Add
import keras as Ks
from keras.layers import Input
from glob import glob
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import tensorflow.keras.backend as K
import tensorflow as tf
import cv2
from skimage import io
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.metrics import Precision, Recall, MeanIoU


from keras.models import Model, Sequential
from keras.layers import Activation, Dense, BatchNormalization, Dropout, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, Input, Reshape
from keras.callbacks import EarlyStopping
from keras import backend as K
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow as tf
import numpy as np
import pandas as pd
import glob
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import cv2
%matplotlib inline
from skimage import io
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from warnings import filterwarnings
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
filterwarnings('ignore')
plt.rcParams["axes.grid"] = False
np.random.seed(101)
from glob import glob

import os
import math
plt.rcParams['figure.figsize'] = [16, 8]
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Conv2D,BatchNormalization,MaxPool2D,Concatenate,Add,Dropout,ReLU,Conv2DTranspose,UpSampling2D
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -U --no-cache-dir gdown --pre
!gdown --id 1-2HGO0znF0JAsgZ_et5Cr_owKVxr_bKZ

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.5.1-py3-none-any.whl size=14951 sha256=222101cecb8e4fbec6d09d40dfdfa961dd132fb171123951d058db790a69ed35
  Stored in directory: /tmp/pip-ephem-wheel-cache-fnb9fs2r/wheels/3d/ec/b0/a96d1d126183f98570a785e6bf8789fca559853a9260e928e1
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-2HGO0znF0JAsgZ_et5Cr_owKVxr_bKZ
To: /content/CT_Da

In [4]:
from zipfile import ZipFile

file_name="/content/CT_Data.zip"

with ZipFile(file_name, 'r') as ziip:
  ziip.extractall()
  print('woow')

woow


In [5]:
mask_files = glob('/content/content/drive/MyDrive/CT_Data/Masks/*')
len(mask_files)

7367

In [6]:
CT = glob('/content/content/drive/MyDrive/CT_Data/CT_images/*')
len(CT)

7367

In [7]:
df = pd.DataFrame({"image_path": CT, "mask_path":mask_files})


In [8]:
Xtrain, Xval = train_test_split(df, test_size=0.15,random_state=101)
Xtest, Xval = train_test_split(Xval, test_size=0.5)
print("Train size is {}, valid size is {} & test size is {}".format(len(Xtrain), len(Xval), len(Xtest)))

Train size is 6261, valid size is 553 & test size is 553


In [9]:
train_ids = list(Xtrain.image_path)
train_mask = list(Xtrain.mask_path)

val_ids = list(Xval.image_path)
val_mask= list(Xval.mask_path)

In [10]:
keep_scale = 0.2
l1l2 = tf.keras.regularizers.l1_l2(l1=0, l2=0.0005)

def resblock(x,level='en_l1',filters=64,keep_scale=keep_scale,l1l2=l1l2,downsample=False,bn_act=True,first_layer=False):
    if downsample:
        if not first_layer:
            x_H = Conv2D(filters,(3,3),(2,2),padding='same',kernel_regularizer=l1l2,name=level+'_Hconv')(x)
            x = Conv2D(filters/2,(3,3),padding='same',kernel_regularizer=l1l2,name=level+'_conv1')(x)
        else:
            x_H = Conv2D(filters,(3,3),(2,2),padding='same',kernel_regularizer=l1l2,name=level+'_Hconv')(x)
            x = Conv2D(filters,(3,3),padding='same',kernel_regularizer=l1l2,name=level+'_conv1')(x)
    else:
        x_H = x
        x = Conv2D(filters,(3,3),padding='same',kernel_regularizer=l1l2,name=level+'_conv1')(x)
    x = BatchNormalization(name=level+'_conv1bn')(x)
    x = ReLU(name=level+'_conv1relu')(x)
    if downsample:
        x = Conv2D(filters,(3,3),(2,2),padding='same',kernel_regularizer=l1l2,name=level+'_conv2')(x)
    else:
        x = Conv2D(filters,(3,3),padding='same',kernel_regularizer=l1l2,name=level+'_conv2')(x)
    x = BatchNormalization(name=level+'_conv2bn')(x)
    x = ReLU(name=level+'_conv2relu')(x)
    x = Add(name=level+'_add')([x_H*keep_scale,x*(1-keep_scale)])
    if bn_act:
        x = BatchNormalization(name=level+'_finalbn')(x)
        x = ReLU(name=level+'_finalrelu')(x)       
    return x


In [11]:
# Encoding layer
# Encoding layer
inputs = tf.keras.Input(shape=(256,256,3),name='input')

layer_0 = Conv2D(64,(7,7),strides=(2,2),padding='same',name='input_conv1',activation=None)(inputs)
layer = BatchNormalization(name='input_conv1bn')(layer_0)
layer = ReLU(name='input_conv1relu')(layer)

## Encoder
layer = resblock(layer,'en_l1',64,keep_scale,l1l2,downsample=True,bn_act=True,first_layer=True)
layer = resblock(layer,'en_l2',64,keep_scale,l1l2,downsample=False,bn_act=True)
encoder_1 = resblock(layer,'en_l3',64,keep_scale,l1l2,downsample=False,bn_act=False)
layer = BatchNormalization(name='en_l3_finalbn')(encoder_1)
layer = ReLU(name='en_l3_finalrelu')(layer)

layer = resblock(layer,'en_l4',128,keep_scale,l1l2,downsample=True,bn_act=True)
layer = resblock(layer,'en_l5',128,keep_scale,l1l2,downsample=False,bn_act=True)
layer = resblock(layer,'en_l6',128,keep_scale,l1l2,downsample=False,bn_act=True)
encoder_2 = resblock(layer,'en_l7',128,keep_scale,l1l2,downsample=False,bn_act=False)
layer = BatchNormalization(name='en_l7_finalbn')(encoder_2)
layer = ReLU(name='en_l7_finalrelu')(layer)

layer = resblock(layer,'en_l8',256,keep_scale,l1l2,downsample=True,bn_act=True)
layer = resblock(layer,'en_l9',256,keep_scale,l1l2,downsample=False,bn_act=True)
layer = resblock(layer,'en_l10',256,keep_scale,l1l2,downsample=False,bn_act=True)
layer = resblock(layer,'en_l11',256,keep_scale,l1l2,downsample=False,bn_act=True)
layer = resblock(layer,'en_l12',256,keep_scale,l1l2,downsample=False,bn_act=True)
encoder_3 = resblock(layer,'en_l13',256,keep_scale,l1l2,downsample=False,bn_act=False)
layer = BatchNormalization(name='en_l13_finalbn')(encoder_3)
layer = ReLU(name='en_l13_finalrelu')(layer)

layer = resblock(layer,'en_l14',256,keep_scale,l1l2,downsample=True,bn_act=True)# 256 instead of 512 concatenate 
layer = resblock(layer,'en_l15',256,keep_scale,l1l2,downsample=False,bn_act=True)#256
layer = resblock(layer,'en_l16',256,keep_scale,l1l2,downsample=False,bn_act=True)#256

## DAC block
b1 = Conv2D(256,(3,3),padding='same',dilation_rate=1,name='dac_b1_conv1',activation=None)(layer)
# b1 = BatchNormalization()(b1)
# b1 = ReLU(name='dac_b1_relu')(b1)

b2 = Conv2D(256,(3,3),padding='same',dilation_rate=3,name='dac_b2_conv1',activation=None)(layer)
b2 = Conv2D(256,(1,1),padding='same',dilation_rate=1,name='dac_b2_conv2',activation=None)(b2)
# b2 = BatchNormalization()(b2)
# b2 = ReLU(name='dac_b2_relu')(b2)

b3 = Conv2D(256,(3,3),padding='same',dilation_rate=1,name='dac_b3_conv1',activation=None)(layer)
b3 = Conv2D(256,(3,3),padding='same',dilation_rate=3,name='dac_b3_conv2',activation=None)(b3)
b3 = Conv2D(256,(1,1),padding='same',dilation_rate=1,name='dac_b3_conv3',activation=None)(b3)
# b3 = BatchNormalization()(b3)
# b3 = ReLU(name='dac_b3_relu')(b3)

b4 = Conv2D(256,(3,3),padding='same',dilation_rate=1,name='dac_b4_conv1',activation=None)(layer)
b4 = Conv2D(256,(3,3),padding='same',dilation_rate=3,name='dac_b4_conv2',activation=None)(b4)
b4 = Conv2D(256,(3,3),padding='same',dilation_rate=5,name='dac_b4_conv3',activation=None)(b4)
b4 = Conv2D(256,(1,1),padding='same',dilation_rate=1,name='dac_b4_conv4',activation=None)(b4)
# b4 = BatchNormalization()(b4)
# b4 = ReLU(name='dac_b4_relu')(b4)

layer = Add(name='dac_add')([layer,b1,b2,b3,b4])
# layer = BatchNormalization(name='dac_bn')(layer)
layer = ReLU(name='dac_relu')(layer)

## RMP block
b1 = MaxPool2D((2,2),strides=(2,2),padding='valid',name='rmp_b1_pool')(layer)
b1 = Conv2D(1,(1,1),padding='valid',name='rmb_b1_conv1',activation=None)(b1)
b1 = Conv2DTranspose(256,(1,1),(2,2),padding='valid',kernel_regularizer=l1l2,output_padding=0,activation=None)(b1)
b1 = tf.image.resize(b1, [8,8], method=tf.image.ResizeMethod.BILINEAR)

b2 = MaxPool2D((3,3),strides=(3,3),padding='valid',name='rmp_b2_pool')(layer)
b2 = Conv2D(1,(1,1),padding='valid',name='rmb_b2_conv1',activation=None)(b2)
b2 = Conv2DTranspose(256,(1,1),(3,3),padding='valid',kernel_regularizer=l1l2,output_padding=0,activation=None)(b2)
b2 = tf.image.resize(b2, [8,8], method=tf.image.ResizeMethod.BILINEAR)

b3 = MaxPool2D((5,5),strides=(5,5),padding='valid',name='rmp_b3_pool')(layer)
b3 = Conv2D(1,(1,1),padding='valid',name='rmb_b3_conv1',activation=None)(b3)
b3 = Conv2DTranspose(256,(1,1),(5,5),padding='valid',kernel_regularizer=l1l2,output_padding=0,activation=None)(b3)
b3 = tf.image.resize(b3, [8,8], method=tf.image.ResizeMethod.BILINEAR)

b4 = MaxPool2D((6,6),strides=(6,6),padding='valid',name='rmp_b4_pool')(layer)
b4 = Conv2D(1,(1,1),padding='valid',name='rmb_b4_conv1',activation=None)(b4)
b4 = Conv2DTranspose(256,(1,1),(6,6),padding='valid',kernel_regularizer=l1l2,output_padding=0,activation=None)(b4)
b4 = tf.image.resize(b4, [8,8], method=tf.image.ResizeMethod.BILINEAR)

layer = Concatenate(name='rmp_concat')([layer,b1,b2,b3,b4])
layer = ReLU(name='rmp_relu')(layer)

layer = Conv2D(256,(1,1),padding='same',kernel_regularizer=l1l2,name='de_l1_conv1',activation=None)(layer)
layer = BatchNormalization(name='de_l1_conv1bn')(layer)
layer = ReLU(name='de_l1_conv1relu')(layer)
layer = Conv2DTranspose(256,(3,3),(2,2),padding='same',kernel_regularizer=l1l2,output_padding=1,name='de_l1_deconv2',activation=None)(layer)
layer = BatchNormalization(name='de_l1_conv2bn')(layer)
layer = ReLU(name='de_l1_deconv2relu')(layer)
layer = Conv2D(256,(3,3),padding='same',kernel_regularizer=l1l2,name='de_l1_conv3',activation=None)(layer)
layer = Add(name='de_l1_add')([encoder_3*keep_scale,layer*(1-keep_scale)])
layer = BatchNormalization(name='de_l1_conv3bn')(layer)
layer = ReLU(name='de_l1_conv3relu')(layer)

layer = Conv2D(128,(1,1),padding='same',kernel_regularizer=l1l2,name='de_l2_conv1',activation=None)(layer)
layer = BatchNormalization(name='de_l2_conv1bn')(layer)
layer = ReLU(name='de_l2_conv1relu')(layer)
layer = Conv2DTranspose(128,(3,3),(2,2),padding='same',kernel_regularizer=l1l2,output_padding=1,name='de_l2_deconv2',activation=None)(layer)
layer = BatchNormalization(name='de_l2_conv2bn')(layer)
layer = ReLU(name='de_l2_deconv2relu')(layer)
layer = Conv2D(128,(1,1),padding='same',kernel_regularizer=l1l2,name='de_l2_conv3',activation=None)(layer)
layer = Add(name='de_l2_add')([encoder_2*keep_scale,layer*(1-keep_scale)])
layer = BatchNormalization(name='de_l2_conv3bn')(layer)
layer = ReLU(name='de_l2_conv3relu')(layer)

layer = Conv2D(64,(1,1),padding='same',kernel_regularizer=l1l2,name='de_l3_conv1',activation=None)(layer)
layer = BatchNormalization(name='de_l3_conv1bn')(layer)
layer = ReLU(name='de_l3_conv1relu')(layer)
layer = Conv2DTranspose(64,(3,3),(2,2),padding='same',kernel_regularizer=l1l2,output_padding=1,name='de_l3_deconv2',activation=None)(layer)
layer = BatchNormalization(name='de_l3_conv2bn')(layer)
layer = ReLU(name='de_l3_deconv2relu')(layer)
layer = Conv2D(64,(1,1),padding='same',kernel_regularizer=l1l2,name='de_l3_conv3',activation=None)(layer)
layer = Add(name='de_l3_add')([encoder_1*keep_scale,layer*(1-keep_scale)])
layer = BatchNormalization(name='de_l3_conv3bn')(layer)
layer = ReLU(name='de_l3_conv3relu')(layer)

layer = Conv2D(64,(1,1),padding='same',kernel_regularizer=l1l2,name='de_l4_conv1',activation=None)(layer)
layer = BatchNormalization(name='de_l4_conv1bn')(layer)
layer = ReLU(name='de_l4_conv1relu')(layer)
layer = Conv2DTranspose(64,(3,3),(2,2),padding='same',kernel_regularizer=l1l2,output_padding=1,name='de_l4_deconv2',activation=None)(layer)
layer = BatchNormalization(name='de_l4_conv2bn')(layer)
layer = ReLU(name='de_l4_deconv2relu')(layer)
layer = Conv2D(64,(1,1),padding='same',kernel_regularizer=l1l2,name='de_l4_conv3',activation=None)(layer)
layer = Add(name='de_l4_add')([layer_0*keep_scale,layer*(1-keep_scale)])
layer = BatchNormalization(name='de_l4_conv3bn')(layer)
layer = ReLU(name='de_l4_conv3relu')(layer)

layer = Conv2DTranspose(32,(3,3),(2,2),padding='same',kernel_regularizer=l1l2,name='final_deconv1',activation=None)(layer)
layer = BatchNormalization()(layer)
layer = ReLU()(layer)
layer = Conv2D(32,(3,3),padding='same',kernel_regularizer=l1l2,name='final_conv1',activation=None)(layer)
layer = BatchNormalization()(layer)
layer = ReLU()(layer)

#pred=Reshape((256,256))(layer) 
outputs = Conv2D(3,(3,3),padding='same',name='output',activation='sigmoid')(layer)
model = tf.keras.Model(inputs,outputs,name='CE-Net')
print(model.summary())

Model: "CE-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_conv1 (Conv2D)           (None, 128, 128, 64  9472        ['input[0][0]']                  
                                )                                                                 
                                                                                                  
 input_conv1bn (BatchNormalizat  (None, 128, 128, 64  256        ['input_conv1[0][0]']            
 ion)                           )                                                            

In [12]:
loaded_model=model
#pred = Reshape((256,256))(x)
#img_input = Input(shape= (256, 256, 3))

#loaded_model=Model(inputs=img_input, outputs=pred)
adam = tf.keras.optimizers.Adam(lr = 0.01, epsilon = 0.1)

In [15]:
loaded_model.compile(optimizer = adam, 
                  loss = focaltversky, 
                  metrics = [tversky,Recall(), Precision(), MeanIoU(num_classes=3)]
                 )

In [16]:
loaded_model.summary()

Model: "CE-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_conv1 (Conv2D)           (None, 128, 128, 64  9472        ['input[0][0]']                  
                                )                                                                 
                                                                                                  
 input_conv1bn (BatchNormalizat  (None, 128, 128, 64  256        ['input_conv1[0][0]']            
 ion)                           )                                                            

In [14]:
def tversky(ytrue, ypred):
    ypredpos = K.flatten(ypred)
    ytruepos = K.flatten(ytrue)
    truepos = K.sum(ytruepos * ypredpos)
    falseneg = K.sum(ytruepos * (1-ypredpos))
    falsepos = K.sum((1-ytruepos)*ypredpos)
    alpha = 0.7
    smooth=100
    return (truepos + smooth)/(truepos + alpha*falseneg + (1-alpha)*falsepos + smooth)

def focaltversky(ytrue,ypred):
    ypred = tf.cast(ypred, tf.float32)
    ytrue = tf.cast(ytrue, tf.float32)
    
    pt_1 = tversky(ytrue, ypred)
    gamma = 0.75
    return K.pow((1-pt_1), gamma)

def tversky_loss(ytrue, ypred):
    return 1 - tversky(ytrue,ypred)

In [17]:
class DataGenerator(tf.keras.utils.Sequence):
  def __init__(self, ids , mask, image_dir = './', batch_size = 16, img_h = 256, img_w = 256, shuffle = True):
    self.ids = ids
    self.mask = mask
    self.image_dir = image_dir
    self.batch_size = batch_size
    self.img_h = img_h
    self.img_w = img_w
    self.shuffle = shuffle
    self.on_epoch_end()
  def __len__(self):
    return int(np.floor(len(self.ids)) / self.batch_size)

  def __getitem__(self, index):
    indexes = self.indexes[index* self.batch_size : (index+1) * self.batch_size]
    list_ids = [self.ids[i] for i in indexes]
    list_mask = [self.mask[i] for i in indexes]
    X, y = self.__data_generation(list_ids, list_mask)
    return X, y

  def on_epoch_end(self):    
    self.indexes = np.arange(len(self.ids))

    if self.shuffle:
      np.random.shuffle(self.indexes)

  def __data_generation(self, list_ids, list_mask):

    X = np.empty((self.batch_size, self.img_h, self.img_w, 3))
    y = np.empty((self.batch_size, self.img_h, self.img_w, 3))

    for i in range(len(list_ids)):
      img_path = str(list_ids[i])
      mask_path = str(list_mask[i])
      img = io.imread(img_path)
      mask = io.imread(mask_path)
      img = cv2.resize(img,(self.img_h,self.img_w))
      img = np.array(img, dtype = np.float64)
      mask = cv2.resize(mask,(self.img_h,self.img_w))
      mask = np.array(mask, dtype = np.float64)
      img -= img.mean()
      img /= img.std()
      mask -= mask.mean()
      mask /= mask.std()
      X[i,] = img
      y[i,] = mask
    y = (y > 0).astype(int)
    return X, y

train_data = DataGenerator(train_ids, train_mask)
val_data = DataGenerator(val_ids, val_mask)

In [18]:
earlystopping = EarlyStopping(monitor='val_loss',
                              mode='min', 
                              verbose=1, 
                              patience=20
                             )
checkpointer = ModelCheckpoint(filepath="/content/drive/MyDrive/CT/ce_net/ce_net_2/ce_net_1.h5", 
                               verbose=1, 
                               save_best_only=True
                              )
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              mode='min',
                              verbose=1,
                              patience=10,
                              min_delta=0.01,
                              factor=0.2
                             )
filename='/content/drive/MyDrive/CT/ce_net/ce_net_2/history.csv'
history_logger=tf.keras.callbacks.CSVLogger(filename, separator=",", append=True)

In [19]:
from tensorflow.keras.metrics import Precision, Recall, MeanIoU

In [20]:
loaded_model.load_weights("/content/drive/MyDrive/CT/ce_net/ce_net_2/ce_net_1.h5")

In [21]:
def prediction(test, model_seg):
    
    mask, image_id, mask_path =  [], [],[]
    
    for i in test.image_path:
        img = io.imread(i)        
        img = img *1./255.
        img = cv2.resize(img, (256,256))
        img = np.array(img, dtype=np.float64)
        img = np.reshape(img, (1,256,256,3))

        X = np.empty((1,256,256,3))
        
        img = io.imread(i)
        img = cv2.resize(img, (256,256))
        img = np.array(img, dtype=np.float64)
        
        img -= img.mean()
        img /= img.std()
        X[0,] = img
        
        predict = model_seg.predict(X)
        
        image_id.append(i)
        mask.append(predict)
    for i in test.mask_path:
        mask_path.append(i)

    return pd.DataFrame({'image_path': image_id,'predicted_mask': mask,'mask_path':mask_path})

In [ ]:
data=prediction(Xtest,model)

KeyboardInterrupt: ignored

In [ ]:
data.head()

,image_path,predicted_mask,mask_path
0,/content/content/drive/MyDrive/CT_Data/CT_imag...,"[[[[0.50627816], [0.49887162], [0.50293034], [...",/content/content/drive/MyDrive/CT_Data/Masks/6...
1,/content/content/drive/MyDrive/CT_Data/CT_imag...,"[[[[0.5045964], [0.49917394], [0.50214535], [0...",/content/content/drive/MyDrive/CT_Data/Masks/6...
2,/content/content/drive/MyDrive/CT_Data/CT_imag...,"[[[[0.5043893], [0.49921113], [0.5020487], [0....",/content/content/drive/MyDrive/CT_Data/Masks/3...
3,/content/content/drive/MyDrive/CT_Data/CT_imag...,"[[[[0.5037114], [0.49933296], [0.5017323], [0....",/content/content/drive/MyDrive/CT_Data/Masks/4...
4,/content/content/drive/MyDrive/CT_Data/CT_imag...,"[[[[0.50403416], [0.49927494], [0.501883], [0....",/content/content/drive/MyDrive/CT_Data/Masks/6...


In [22]:
loaded_model.fit(train_data, 
                  epochs = 250,
                  initial_epoch=58,
                  validation_data = val_data,
                  callbacks = [checkpointer ,earlystopping, reduce_lr,history_logger]
                 )

Epoch 59/250
391/391 [==============================] - ETA: 0s - loss: 0.9354 - tversky: 0.1271 - recall: 0.0000e+00 - precision: 0.0000e+00 - mean_io_u: 0.4997
Epoch 59: val_loss improved from inf to 0.95818, saving model to /content/drive/MyDrive/CT/ce_net/ce_net_2/ce_net_1.h5
391/391 [==============================] - 164s 333ms/step - loss: 0.9354 - tversky: 0.1271 - recall: 0.0000e+00 - precision: 0.0000e+00 - mean_io_u: 0.4997 - val_loss: 0.9582 - val_tversky: 0.1007 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_mean_io_u: 0.4997 - lr: 0.0100
Epoch 60/250
391/391 [==============================] - ETA: 0s - loss: 0.9373 - tversky: 0.1242 - recall: 0.0000e+00 - precision: 0.0000e+00 - mean_io_u: 0.4997
Epoch 60: val_loss improved from 0.95818 to 0.95574, saving model to /content/drive/MyDrive/CT/ce_net/ce_net_2/ce_net_1.h5
391/391 [==============================] - 134s 343ms/step - loss: 0.9373 - tversky: 0.1242 - recall: 0.0000e+00 - precision: 0.0000e+00 - mean_io

KeyboardInterrupt: ignored

In [40]:
loaded_model.predict(resized_image)

ValueError: ignored

In [ ]:
train_generator_args = dict(rotation_range=0.2,
                            width_shift_range=0.05,
                            height_shift_range=0.05,
                            shear_range=0.05,
                            zoom_range=0.05,
                            horizontal_flip=True,
                            fill_mode='nearest')
train_gen = train_generator(df_train, BATCH_SIZE,
                                train_generator_args,
                                target_size=(im_height, im_width))
    
test_gener = train_generator(df_val, BATCH_SIZE,
                                dict(),
                                target_size=(im_height, im_width))
    
model = unet(input_size=(im_height, im_width, 3))



decay_rate = learning_rate / EPOCHS
opt = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=decay_rate, amsgrad=False)
model.compile(optimizer=opt, loss=dice_coef_loss, metrics=["binary_accuracy", iou, dice_coef])

callbacks = [ModelCheckpoint('unet_brain_mri_seg.hdf5', verbose=1, save_best_only=True)]

history = model.fit(train_gen,
                    steps_per_epoch=len(df_train) / BATCH_SIZE, 
                    epochs=EPOCHS, 
                    callbacks=callbacks,
                    validation_data = test_gener,
                    validation_steps=len(df_val) / BATCH_SIZE)